## Scénario
Aujourd’hui est un grand jour, vous commencez votre mission chez BottleNeck, un marchand de vin très prestigieux. 
Le responsable du projet vous explique les enjeux de votre première mission :

 “Actuellement, pour gérer nos ressources, nos clients, etc., on utilise un ERP qui n’est absolument pas relié à notre site de vente en ligne. Pour être tout à fait honnête, les outils en place sont vraiment artisanaux et dans ces conditions, la gestion des stocks est vraiment complexe et notre visibilité en termes d’analyse des ventes sur le Net est vraiment réduite, car très peu de personnes ont accès au back-office. En attendant une solution plus centralisée, un rapprochement entre les 2 bases, même manuel, pourrait être très utile…

Ta première mission se passe en 3 points.

* Premièrement, j’ai besoin que tu rapproches deux exports : un export de l’ERP contenant les références produit, leur prix de vente et leur état de stock, et un export d’une table de l’outil de CMS contenant les informations des produits commercialisés en ligne (nom, description, nombre de ventes...), il sont présentées pers les deux fichiers (1.xls, 2.xlsx). En plus de ces 2 exports, tu vas bénéficier d’une aide précieuse car Sylvie, notre ancienne stagiaire, a réalisé un travail de fourmi. Elle a créé un tableau Excel qui permet d’établir le lien entre la référence du produit dans l’ERP (product_id) et la référence du même produit dans la base de la boutique en ligne (SKU) les données sont présentées par le fichier 3.xls ( En fait, la colonne id_web dans son fichier correspond au SKU des produits dans la boutique en ligne.)

* Deuxièmement, une fois le rapprochement effectué, je souhaiterais avoir le chiffre d’affaires par produit, ainsi que le total du chiffre d’affaires réalisé en ligne.

* Troisièmement et pour finir, je me demande s’il n’y a pas eu des erreurs de saisie dans certains prix des produits. J'aimerais que tu effectues une analyse sur cette variable afin de détecter d’éventuelles valeurs aberrantes, de les lister et d’en faire une représentation graphique pour plus de lisibilité.

**Nous voudrions présenter tes résultats lors de la prochaine réunion de COPIL. Cela permettrait de montrer nos progrès. Je peux t’épargner le travail d’une présentation par slides, il me faut juste un notebook que tu présenteras à l’assemblée pour expliquer ta démarche.**”

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [112]:
df_erp = pd.read_excel('data/1.xlsx')
df_inter = pd.read_excel('data/2.xlsx')
df_sku = pd.read_excel('data/3.xlsx')

C:\Users\utilisateur\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [113]:
df_erp.head()

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


In [114]:
df_merge = pd.merge(df_erp, df_inter, on='product_id', how='inner')
df_merge.head()

,product_id,onsale_web,price,stock_quantity,stock_status,id_web
0,3847,1,24.2,0,outofstock,15298
1,3849,1,34.3,0,outofstock,15296
2,3850,1,20.8,0,outofstock,15300
3,4032,1,14.1,0,outofstock,19814
4,4039,1,46.0,0,outofstock,19815


In [115]:
df_merge.rename(columns={'id_web': 'sku'}, inplace=True)
df_sku.columns

Index(['sku', 'virtual', 'downloadable', 'rating_count', 'average_rating',
       'total_sales', 'tax_status', 'tax_class', 'post_author', 'post_date',
       'post_date_gmt', 'post_content', 'post_title', 'post_excerpt',
       'post_status', 'comment_status', 'ping_status', 'post_password',
       'post_name', 'post_modified', 'post_modified_gmt',
       'post_content_filtered', 'post_parent', 'guid', 'menu_order',
       'post_type', 'post_mime_type', 'comment_count'],
      dtype='object')

In [116]:
df_merge = pd.merge(df_merge, df_sku, on='sku', how='inner')
df_merge.drop_duplicates(subset=['sku'], keep='first', inplace=True)

In [117]:
df_merge.head

<bound method NDFrame.head of       product_id  onsale_web  price  stock_quantity stock_status      sku  \
0           3847           1   24.2               0   outofstock    15298   
2           3849           1   34.3               0   outofstock    15296   
4           3850           1   20.8               0   outofstock    15300   
6           4032           1   14.1               0   outofstock    19814   
8           4039           1   46.0               0   outofstock    19815   
...          ...         ...    ...             ...          ...      ...   
9153        6930           1    8.4              83      instock    16135   
9155        7023           1   27.5              15      instock    15891   
9157        7025           1   69.0               2      instock    15887   
9159        7247           1   54.8              23      instock  13127-1   
9161        7338           1   16.3              45      instock    16230   

      virtual  downloadable  rating_count  av

<span style="color: orange;">Maintenant que nous avons fusionnés nos Dataframe, nous allons rajouter une nouvelle colonne **Chiffre d'affaire**</span>

In [118]:
df_merge["CA"] = df_merge["total_sales"] * df_merge["price"]
df_merge["CA"] = df_merge["CA"].astype(float)

In [119]:
df_merge["CA"]

0       145.2
2         0.0
4         0.0
6        42.3
8         0.0
        ...  
9153     42.0
9155      0.0
9157      0.0
9159      0.0
9161      0.0
Name: CA, Length: 715, dtype: float64

In [120]:
ca_total = df_merge['CA'].sum()

print(f"Le Chiffre d'affaire total est de {ca_total} €")

Le Chiffre d'affaire total est de 70568.6 €


In [121]:
df_merge.to_excel('data/merge.xlsx', index=False)